In [2]:
#!pip install pymemcache

In [10]:
#!apt install -y memcached

Reading package lists... Done
Building dependency tree       
Reading state information... Done
memcached is already the newest version (1.5.22-2ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 54 not upgraded.


In [5]:
import openslide
from pymemcache.client.base import PooledClient
from pymemcache import serde

import numpy as np

tile_size=2_048

mask_image=np.random.uniform(size=(tile_size,tile_size))>0.5
wsi=openslide.OpenSlide('/opt/QuickAnnotator/quickannotator/data/test_ndpi/TCGA-23-2072-01Z-00-DX1.478243FF-BFF0-48A4-ADEA-DE789331A50E.svs')


In [8]:
io_image=np.array(wsi.read_region((0,0),0,(tile_size,tile_size)))[:,:,:3]

In [20]:
#!memcached -m 10240 -I 20m -vv -u www-data &

OSError: Background processes not supported.

In [21]:
import pickle
import zlib

In [46]:
#--- this is probably the "safest" way to compress stuff into the cache -- but its not the most efficient for iamges
def serialize(data):
    # Pickle the data and compress it using zlib
    return zlib.compress(pickle.dumps(data))
# Custom deserialization function
def deserialize(data):
    # Decompress the data and unpickle it
    return pickle.loads(zlib.decompress(data))

In [23]:
compressed_tuple=serialize((io_image,mask_image))

In [25]:
len(compressed_tuple)

933348

In [27]:
import numpy as np
from PIL import Image
import io

In [49]:
#--- instead can use jpg, reduce storage costs by 90%
def compress_to_jpeg(matrix):
    # Convert NumPy matrix to a PIL Image
    image = Image.fromarray(matrix.astype(np.uint8))
    # Save the image to a BytesIO object as JPEG
    with io.BytesIO() as output:
        image.save(output, format="JPEG")
        jpeg_bytes = output.getvalue()  # Get the byte data
    return jpeg_bytes

# Function to decompress JPEG bytes back to a NumPy array
def decompress_from_jpeg(jpeg_bytes):
    # Open the byte data as an image using PIL
    with io.BytesIO(jpeg_bytes) as input:
        image = Image.open(input)
        # Convert image back to NumPy array
        matrix = np.array(image)
    return matrix

In [40]:
compressed_images = [compress_to_jpeg(i) for i in (io_image,mask_image)]


In [48]:
len(compressed_images[0])+len(compressed_images[1])

133649

In [43]:
decompressed_images = [decompress_from_jpeg(i) for i in compressed_images]

In [39]:
client = PooledClient(('localhost', 11211),serde=serde.pickle_serde, max_pool_size=4)
key = "test_1_23"
client.set(key, (io_image,mask_image))


True

In [19]:
%%timeit
retval=client.get(key)

11.5 ms ± 583 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
